# Data engineering

In [1]:
from __future__ import print_function
from pandas_profiling import ProfileReport
from dateutil.parser import parse
from ipywidgets import interact

import matplotlib.pyplot as plt
import multiprocessing as mp
import ipywidgets as widgets
import plotly.express as px
import missingno as msno
import pandas as pd
import unicodedata
import numpy as np
import hashlib
import glob
import re

## Variables

In [2]:
RAW_DATA = './downloads/raw'
PPD_DATA = './downloads/pre-processed'

## Functions

Replace function

- df: input dataframe
- rules: substitution rules `{index: {'cols': columns, 'replaces': {value: new_value}, 'regex': bool}}` eg:
    - `{0: {'cols': 'Gender', 'replaces': {None:0.0}, 'regex': True}}`

In [3]:
def replace(df: pd.DataFrame, rules: {}) -> pd.DataFrame:    
    for rule in rules.values():
        cols = rule['cols']
        df[cols] = df[cols].replace(rule['replaces'], regex=rule['regex'])
    return df

## Visualization

In [4]:
df = pd.read_csv(f'{PPD_DATA}/01_07-05_10-wcoords.csv', low_memory=False)
df

,Adinamia (fraqueza),Alteração de ausculta pulmonar,Alteração na radiologia de tórax,Apresenta sintomas?,Bairro,CEP,Cefaleia (dor de cabeça),"Complemento ( Condomínio, edifício, apto)",Conjuntivite,Coriza,...,Situação de saúde do paciente no momento da notificação:*,Tosse,[outros] Condições:Selecionar todas morbidades pertinentes.,[outros] Selecione os sinais clínicos observados:,addr,error_message,lat,lon,results,status
0,NaN,NaN,NaN,2.0,Piatã,41650195.0,NaN,Stupendo 1704B,NaN,NaN,...,NaN,NaN,NaN,NaN,"Rua da Gratidão , Stupendo 1704B, Piatã , Salv...",NaN,-12.937704,-38.383711,"[{'address_components': [{'long_name': '232', ...",OK
1,NaN,NaN,NaN,1.0,Horto Florestal,40295010.0,NaN,Edfo Mansão Horto Florestal,NaN,1.0,...,NaN,NaN,NaN,1.0,"Waldemar Falcão ,, Edfo Mansão Horto Florestal...",NaN,-13.001480,-38.489633,"[{'address_components': [{'long_name': '1422',...",OK
2,NaN,NaN,NaN,2.0,Pituba,41810105.0,NaN,Edf Luane Guimarães apto 1101,NaN,NaN,...,NaN,NaN,NaN,NaN,"Marechal Andréa, Edf Luane Guimarães apto 1101...",NaN,-12.996560,-38.451451,"[{'address_components': [{'long_name': '448', ...",OK
3,NaN,NaN,NaN,2.0,Arembepe,42835000.0,NaN,Casa,NaN,NaN,...,NaN,NaN,NaN,NaN,"Eduardo Pinto , Casa, Arembepe, Camaçari, Bahi...",NaN,-12.762831,-38.180420,[{'address_components': [{'long_name': 'Rua Ed...,OK
4,NaN,NaN,NaN,2.0,Camaçari de dentro,42806090.0,NaN,Casa,NaN,NaN,...,NaN,NaN,NaN,NaN,"Ana Galvão , Casa, Camaçari de dentro , Camaça...",NaN,-12.704338,-38.334264,[{'address_components': [{'long_name': 'Rua An...,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,45.522208,-122.986328,"[{'address_components': [{'long_name': '125', ...",OK
19637,NaN,NaN,NaN,2.0,Gravito,48110000.0,NaN,Sítio do Sinho,NaN,NaN,...,NaN,NaN,NaN,NaN,"Oscar Pereira de Souza , Sítio do Sinho, Gravi...",NaN,-12.370478,-38.393546,[{'address_components': [{'long_name': 'Rua Os...,OK
19638,NaN,NaN,NaN,2.0,Pero Vaz,40335180.0,NaN,Casa,NaN,NaN,...,NaN,NaN,NaN,NaN,"Rua Nossa Senhora das Graças, Casa, Pero Vaz, ...",NaN,-12.951535,-38.488931,[{'address_components': [{'long_name': 'Rua No...,OK
19639,NaN,NaN,NaN,2.0,Buraquinho,42709269.0,NaN,Cond Gileade d12,NaN,NaN,...,NaN,NaN,NaN,NaN,"Arlete Souza Costa , Cond Gileade d12, Buraqui...",NaN,-12.876423,-38.291201,"[{'address_components': [{'long_name': '349', ...",OK


In [5]:
rules = {
    0: {
        'cols': 'Sexo',
        'replaces': {1: 'Masculino', 2: 'Feminino'},
        'regex': False
    },
    1: {
        'cols': 'Raça',
        'replaces': {
            1: 'amarela',
            2: 'branca',
            3: 'indigena',
            4: 'parda',
            5: 'preta'
        },
        'regex': False
    },
    2: {
        'cols': [
            'Apresenta sintomas?',
            'Paciente foi hospitalizado?',
            'Paciente foi submetido a ventilação mecânica?',
        ],
        'replaces': {1: 'sim', 2: 'não', np.nan: 'nan'},
        'regex': False
    },
    3: {
        'cols': 'RESULTADO',
        'replaces': {1: 'Positivo', 2: 'Negativo', 9: 'Inconclusivo'},
        'regex': False
    }
}

df = replace(df, rules)

In [6]:
# [c for c in df.columns]

In [7]:
symptoms = [
    'Adinamia (fraqueza)',
    'Cefaleia (dor de cabeça)',
    'Coriza',
    'Diarreia',
    'Dificuldade de respirar',
    'Dispneia/Taquipneia',
    'Dor de garganta',
    'Exsudato faríngeo',
    'Febre',
    'Irritabilidade/confusão',
    'Neoplasia (tumor sólido ou hematológico)',
    'Náusea/vômitos',
    'Tosse',
]
comorbidities = [
    'Alteração de ausculta pulmonar',
    'Alteração na radiologia de tórax',
    'Conjuntivite',
    'Diabetes',
    'Doença Respiratória crônica',
    'Doença cardiovascular, incluindo hipertensão',
    'Doença hepática',
    'Doença neurológica crônica ou neuromuscular',
    'Doença renal',
    'Doença respiratória crônica',
    'Gestante',
    'Gestante de alto risco',
    'Imunodeficiência',
    'Imunossupressão',
    'Infecção pelo HIV',
    'Portador de alterações cromossômicas',
    'Portador de doenças cromossômicas',
    'Sem morbidades',
]

_vall = [*symptoms, *comorbidities]
_wall = {
    v: widgets.Checkbox(value=False, description=v)
    for v in _vall
}

In [9]:
@interact(
    result=widgets.Dropdown(options=['All', 'Positivo', 'Negativo', 'Inconclusivo']),
    **_wall,
    range_color=widgets.IntRangeSlider(
        value=[0,0],
        min=0,
        max=1000,
        step=1,
        continuous_update=False,
        orientation='horizontal'
    ),
    radius=widgets.IntSlider(
        value=1,
        min=1,
        max=50,
        step=1,
        continuous_update=False,
        orientation='horizontal'
    ),
    _all=widgets.Checkbox(value=True, description='All')
)
def show(result: str, range_color: list, radius: int, _all: bool, **_vall):
    aux = df.copy()
    
    if result != 'All':
        aux = aux[aux['RESULTADO'] == result]

    if _all is False:
        _f = [False] * len(aux)
        for p in _vall:
            if _vall[p] is True:
                _f |= aux[p] == 1
            else:
                _f |= aux[p] == 0
        aux = aux[_f]

    fig = px.density_mapbox(
        aux,
        lat='lat',
        lon='lon',
        #z='Confirmed',
        radius=radius, # 1
        center=dict(lat=-12.8, lon=-38),
        zoom=8,
        hover_data=[
            'Raça',
            'Sexo',
            'Apresenta sintomas?',
            'Paciente foi hospitalizado?',
            'Paciente foi submetido a ventilação mecânica?',
            'Nome do hospital de internação:',
            'Nome do paciente (completo)',
            'Ocupação',
            'Nome da Empresa',
            'Nome do caso fonte:',
            'Qual a sua ocupação',
            'DATA DA COLETA',
            'RESULTADO',
        ],
        mapbox_style="carto-positron", # stamen-terrain
    #     animation_frame='DATA DA COLETA',
        range_color=range_color,
        color_continuous_scale=px.colors.sequential.solar_r,
       title=f'Geolocation RT/PCR Test - {len(aux)} records'
    )
    # fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
    fig.show()


interactive(children=(Dropdown(description='result', options=('All', 'Positivo', 'Negativo', 'Inconclusivo'), …